In [10]:
#用益信托网信息爬取并导入excel

import requests
import json
import pandas
from lxml import etree
import time

url = 'http://www.yanglee.com/Action/ProductAJAX.ashx?mode=statistics&pageSize=40&pageIndex={}&conditionStr=producttype%3A3&start_released=&end_released=&orderStr=1&ascStr=ulup&_=1552274218437'

url1 = 'http://www.yanglee.com/Product/Detail.aspx?id='

shuju = []

headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Connection': 'close',
    'Cookie': 'safedog-flow-item=4F9739A28BFFB7B6774E420EF8B9BEAA; UM_distinctid=1684b5c101b205-0737359e6372ae-b781636-1fa400-1684b5c101c27; CNZZDATA1309589=cnzz_eid%3D1255970783-1547450167-http%253A%252F%252Fwww.yanglee.com%252F%26ntime%3D1547450167',
    'Host': 'www.yanglee.com',
    'Referer': 'http://www.yanglee.com/Product/Index.aspx',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}


def get_page(p):
    
    try:
        
        url0 = url.format(p)
    
        r = requests.get(url0, headers = headers, timeout = 12)

        r.encoding = r.apparent_encoding

        return r.json()
    
    except Exception as e1:
        
        print('a', str(e1))
    

def get_info(page):
    
    for n in range(0, 40):     #一页40个产品
    
        try:
            
            #print(d.get('result')[n].get('ID'))
    
            url2 = url1 + page.get('result')[n].get('ID')
        
            info = {}
    
            r2 = requests.get(url2, timeout = 12)
    
            r2.encoding = r2.apparent_encoding
    
            #print(r2.text)
    
            html = etree.HTML(r2.text)
    
            info['产品名称'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[1]/td[2]/text()'))
    
            info['发行机构'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[3]/td[2]/text()'))
    
            info['发行地'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[4]/td[2]//text()'))
        
            info['收益分配'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[4]/td[4]//text()'))
    
            info['发行时间'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[5]/td[2]/text()'))
    
            info['发行规模'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[5]/td[4]/text()'))
        
            info['产品期限'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[7]/td[2]/text()'))
    
            info['期限类型'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[7]/td[4]/text()'))
        
            info['预计收益'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[8]/td[2]/text()'))
    
            info['收益类型'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[8]/td[4]/text()'))
    
            info['投资方式'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[9]/td[2]/text()'))
    
            info['资金托管行'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[9]/td[4]/text()'))
    
            info['投资领域'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[10]/td[2]/text()'))
        
            info['投资项目所在地'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[10]/td[4]/text()'))

            info['产品特点'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[11]/td[2]/p/text()'))
    
            info['其他相关信息'] = " ".join(html.xpath('//*[@id="procon1"]/table/tr[12]/td[2]/p/text()'))
    
            info['融资方'] = " ".join(html.xpath('//*[@id="t3"]/p/text()[1]'))
        
            #info['融资方简介'] = " ".join(html.xpath('//*[@id="t3"]/p/text()[2]'))
        
            info['资金用途'] = " ".join(html.xpath('//*[@id="t4"]/p/text()'))
        
            info['还款来源'] = " ".join(html.xpath('//*[@id="t5"]/p/text()'))
        
            info['风控措施'] = " ".join(html.xpath('//*[@id="t6"]/p/text()'))
        
            info['管理人'] = " ".join(html.xpath('//*[@id="t7"]/p/text()[1]'))
        
            #info['资产管理人简介'] = " ".join(html.xpath('//*[@id="t7"]/p/text()[2]'))
        
            info['链接'] = url2
        
            print("已爬取第【{}】个产品信息，链接：".format( n+1),url2)
    
            yield info
        
        except Exception as e2:
            
            print('b', str(e2))
        
        
def write_to_excel(shuju):
    df = pandas.DataFrame(shuju)
    df.to_excel('用益私募爬虫20190428v2.xlsx')
        
def main():
    print("执行开始".center(150//2,'-'))
    start = time.perf_counter()
    sp = 1
    ep = 11
    for p in range(sp, ep):
        print("正在爬取第【{}】页数据".format(p))
        page = get_page(p)
        mores = get_info(page)
        for more in mores:
            shuju.append(more)
        print("已完成第【{}】页数据爬取".format(p))
        print("----------------------------------------------------------------------------")
    write_to_excel(shuju)
    dur = time.perf_counter() - start
    print("数据爬取总耗时{:.2f}s".format(dur))
    print("\n"+"执行结束".center(150//2,'-'))
    
if __name__ == '__main__':
    main()


------------------------------------执行开始-----------------------------------
正在爬取第【1】页数据
已爬取第【1】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283622548
已爬取第【2】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283642399
已爬取第【3】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283645728
已爬取第【4】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283617726
b HTTPConnectionPool(host='www.yanglee.com', port=80): Read timed out. (read timeout=12)
已爬取第【6】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283624118
已爬取第【7】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283618958
已爬取第【8】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283642874
已爬取第【9】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283612539
已爬取第【10】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283649841
已爬取第【11】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283668364
已爬取第【12】个产品信息，链接： http://www.ya

已爬取第【20】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283638545
已爬取第【21】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283612922
已爬取第【22】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283649442
已爬取第【23】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283698696
已爬取第【24】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283682439
已爬取第【25】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283641657
已爬取第【26】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283641686
已爬取第【27】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283623732
已爬取第【28】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283612564
已爬取第【29】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283684112
已爬取第【30】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283647845
已爬取第【31】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283656923
已爬取第【32】个产品信息，链接： http://www.yanglee.com

已爬取第【1】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283668832
已爬取第【2】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283643469
已爬取第【3】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283635555
已爬取第【4】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283654959
已爬取第【5】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283652251
已爬取第【6】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283625681
已爬取第【7】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283645171
已爬取第【8】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283611685
已爬取第【9】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283676551
已爬取第【10】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283619698
已爬取第【11】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283681975
已爬取第【12】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283657398
已爬取第【13】个产品信息，链接： http://www.yanglee.com/Product/

已爬取第【22】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283651488
已爬取第【23】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283644217
已爬取第【24】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283635962
已爬取第【25】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283675713
已爬取第【26】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283639775
已爬取第【27】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283693451
已爬取第【28】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283658838
已爬取第【29】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283666761
已爬取第【30】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283633538
已爬取第【31】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283643383
已爬取第【32】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283686254
已爬取第【33】个产品信息，链接： http://www.yanglee.com/Product/Detail.aspx?id=100012283683352
已爬取第【34】个产品信息，链接： http://www.yanglee.com

数据爬取总耗时1137.27s

------------------------------------执行结束-----------------------------------
